<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/BERT_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installs

In [4]:
!pip install transformers datasets evaluate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [5]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.5 MB/s eta 0:00:00


In [6]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip uninstall -y transformers
!pip install transformers==4.28.0

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-saopdxh1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-saopdxh1
  Resolved https://github.com/huggingface/accelerate to commit 9a86a49f72f5c9c0ca521b903fb1a44fa8ee8f53
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproj

## Token hugging face

In [7]:
from huggingface_hub import notebook_login

notebook_login()

## Mounting

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Mounted at /content/gdrive


## Dataset

In [6]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}Official/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df

df = pd.DataFrame(df)
dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)

# dataset = dataset.rename_column("code_frames", "label")
dataset = dataset.rename_column("annotations", "text")

dataset

DatasetDict({
    train: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 37439
    })
    test: Dataset({
        features: ['code_frames', 'text', 'label'],
        num_rows: 9360
    })
})

## Prep 

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased",
                                          do_lower_case=False)


In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, 
    max_length=128)

In [9]:
mfc_tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/37439 [00:00<?, ? examples/s]

Map:   0%|          | 0/9360 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
id2label, label2id = {},{}
for x in range(15):
    id2label[x] = x
        # {0: "NEGATIVE", 1: "POSITIVE"}
    label2id[x] = x
        # {"NEGATIVE": 0, "POSITIVE": 1}

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=15, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

## Training

In [18]:
training_args = TrainingArguments(
    output_dir="Bert-trial",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mfc_tokenized["train"],
    eval_dataset=mfc_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/Bert-trial is already a clone of https://huggingface.co/jmLuis/Bert-trial. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.056600,0.934590,0.716453
2,0.774700,0.878050,0.732372
3,0.539900,0.896228,0.740705
4,0.383200,0.974471,0.744444
5,0.276500,1.087449,0.739637
6,0.200400,1.166801,0.742521
7,0.158200,1.229310,0.746261
8,0.130000,1.311914,0.744338
9,0.107100,1.349917,0.748504
10,0.090800,1.373488,0.747329


TrainOutput(global_step=11700, training_loss=0.38943232919415854, metrics={'train_runtime': 8952.2159, 'train_samples_per_second': 41.821, 'train_steps_per_second': 1.307, 'total_flos': 1.7475327949322484e+16, 'train_loss': 0.38943232919415854, 'epoch': 10.0})

In [20]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/May23_06-27-47_b0d428ad89d4/events.out.tfevents.1684823269.b0d428ad89d4.195.0:   0%|         …

To https://huggingface.co/jmLuis/Bert-trial
   e6654cd..3bf1cfe  main -> main

   e6654cd..3bf1cfe  main -> main

To https://huggingface.co/jmLuis/Bert-trial
   3bf1cfe..57ac3af  main -> main

   3bf1cfe..57ac3af  main -> main



'https://huggingface.co/jmLuis/Bert-trial/commit/3bf1cfedf8342de533dac196663939f14f2e4824'

# Inference

In [9]:
from transformers import AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer


In [10]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = 'Immigrants without HOPE need help entering college'

In [11]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
inputs = tokenizer(text, return_tensors="pt")

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")
with torch.no_grad():
    logits = model(**inputs).logits

In [13]:
predicted_class_id = logits.argmax().item()
out = model.config.id2label[predicted_class_id]
out+1

10

In [23]:
tokenizer = AutoTokenizer.from_pretrained("jmLuis/Bert-trial")
model = AutoModelForSequenceClassification.from_pretrained("jmLuis/Bert-trial")


def infer_bert(row):
    inputs = tokenizer(row['Text'], return_tensors="pt",  truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    out = model.config.id2label[predicted_class_id]
    return int(out)+1  #adding onw because original label is not zero index, however training is.

## applying in curated dataset

In [16]:
import pandas as pd
curated_df = pd.read_csv(f'{dir}Official/curated_Dataset.csv', usecols=["Annotate", "Text"], header=0)

curated_df

,Annotate,Text
0,NaN,"DOH logs 1,920 new COVID-19 cases, highest dai..."
1,NaN,"The Philippines on Sunday registered 1,920 ne..."
2,NaN,"According to GMA Integrated News Research, th..."
3,NaN,"It is the fourth straight day of more than 1,..."
4,NaN,Data from the Department of Health's COVID-19...
...,...,...
132092,NaN,"""The Philippines and China also discussed new ..."
132093,NaN,"On Thursday, Lazaro started chairing with Chin..."
132094,NaN,"The conduct of the BCM, a confidence-building ..."
132095,NaN,"In an earlier pronouncement, Lazaro asserted t..."


In [17]:
curated_df.shape

(132097, 2)

In [18]:
# save=0
# for i, row in curated_df.iterrows():
#     curated_df.at[i,'Annotate'] = infer_bert(row)
#     if(i%100==0):
#         print(i)
#     if (i%2000==0):
#         print('saved', save)
#         curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
#         save+=1


0
saved 0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
saved 1
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
saved 2
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
saved 3
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
saved 4
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
saved 5
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
saved 6
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
saved 7
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
saved 8
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17

Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:3                                                                              │
│ in infer_bert:8                                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:1562 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1559 │   │   """                                                                               │
│   1560 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1561 │   │                                                                                     │
│ ❱ 1562 │   │   outputs = self.bert(                                                              │
│   1563 │   │   │   input_ids,                                                                    │
│   1564 │   │   │   attention_mask=attention_mask,                                                │
│   1565 │   │   │   token_type_ids=token_type_ids,                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/bert/modeling_bert.py:1013 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1010 │   │   # and head_mask is converted to shape [num_hidden_layers x batch x num_heads x s  │
│   1011 │   │   head_mask = self.get_head_mask(head_mask, se

In [ ]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (642 > 512). Running this sequence through the model will result in indexing errors
# RuntimeError: The size of tensor a (642) must match the size of tensor b (512) at non-singleton dimension 1

In [19]:
curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

In [22]:
curated_df.iloc[:29953].to_csv(f'{dir}BERT-curated/Bert_curated_dataset-cut.csv', index=False)

In [ ]:
save=15
for i, row in curated_df.iloc[29900:].iterrows():
    curated_df.at[i,'Annotate'] = infer_bert(row)
    if(i%100==0):
        print(i)
    if (i%2000==0):
        print('saved', save)
        curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-{save}.csv', index=False)
        save+=1


29900
30000
saved 15
30100
30200
30300
30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
saved 16
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
saved 17
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
saved 18
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
saved 19
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
saved 20
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
saved 21
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100


In [ ]:
curated_df.to_csv(f'{dir}BERT-curated/Bert_curated_dataset-last.csv', index=False)

# Others

In [ ]:
#REference fine tuning
# https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/text_classification.ipynb